# Fase 0: <i>Recopilación de datos</i>
## <b>Obtención de los vuelos y precios de <i>ryanair.com</i></b>

&emsp;&emsp;Con este archivo se procede a recopilar la información de los vuelos y precios de la página <b>ryanair.com</b>.

**1.IMPORTACIÓN DE LIBRERIAS**

In [1]:
import requests
from time import sleep, strftime
import random
from random import randint
from bs4 import BeautifulSoup
from selenium import webdriver
import shutil
import os
import pandas as pd
from random import randint
from time import sleep, strftime
from datetime import *

**2.CARGA DE LOS DATOS**

Se cargan los datos para obtener las rutas (origenes y destinos) que existen actualmente en la compañía aérea.

In [ ]:
df = pd.read_csv('../../data/ryanair.csv')

**3. FILTRADO DE ORIGENES**

Filtramos los origenes con los que va a trabajar la aplicación en su primera versión (Madrid, Barcelona)

In [ ]:
df = df[(df['Source Airport']=='MAD')|(df['Source Airport']=='BCN')]

**4. LISTA DESTINOS Y ORIGENES**

Se crean listas de origenes y destinos que iterarán en el proceso de Web Scraping.

In [39]:
destinos = list(df['Destination Airport'].values)
origen = list(df['Source Airport'].values)

**5. WEB SCRAPING**

A continuación se presenta el código de web Scraping para la web Ryanair.com

In [43]:
#Driver de chrome 
chrome_driver = "chromedriver.exe"

In [54]:
#Función para extraer vuelos
def GetVuelos(soup, url, origen, destino, fecha_ida):
    dfAux = pd.DataFrame(columns=['CAPTURA', 'FUENTE', 'ORIGEN', 'DESTINO', 'SALIDA', 'LLEGADA', 'PRECIO', "URL", "AEROLINEA"])
    listaVuelos = soup.find_all("div", class_='card-wrapper')
    for i in range(len(listaVuelos)):
        vuelo = listaVuelos[i]
        hora = vuelo.find('div', {'data-ref':'flight-segment.departure'}).find('span', class_='h2').text.strip()
        hora_llegada = vuelo.find('div', {'data-ref':'flight-segment.arrival'}).find('span', class_='h2').text.strip()
        salida = fecha_ida +' '+ hora
        llegada = fecha_ida +' '+ hora_llegada
        try:
            precio = float(vuelo.find('span', class_='price-value h2 text-700').text.replace('€','').replace(',','.').strip())
        except:
            precio = float(vuelo.find('span', class_='price-value h2 text-700 price-value--discounted').text.replace('€','').replace(',','.').strip())
        dfAux = dfAux.append({"CAPTURA": datetime.now().strftime('%Y-%m-%d'), "FUENTE": "Ryanair", "ORIGEN":origen, "DESTINO":destino, "SALIDA":salida, "LLEGADA":llegada,"PRECIO":precio, "URL":url, "AEROLINEA":"Ryanair"}, ignore_index = True)
    return dfAux

In [57]:
%%time
df_vuelos = pd.DataFrame(columns=['CAPTURA', 'FUENTE', 'ORIGEN', 'DESTINO', 'SALIDA_IDA', 'LLEGADA_IDA', 'SALIDA_VUELTA', 'LLEGADA_VUELTA','PRECIO', "URL", "AEROLINEA"])

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("--incognito")
navegador = webdriver.Chrome(executable_path=chrome_driver,options=chrome_options)

fecha_ida = datetime(2020,10,1)
fecha_vuelta = fecha_ida + timedelta(days=7)
incrementoDia = timedelta(days=1)
texto_fecha_ida = fecha_ida.strftime('%Y-%m-%d')
texto_fecha_vuelta = fecha_vuelta.strftime('%Y-%m-%d')

for k in range(90):
    for j, i in zip(origen,destinos):
        df_ida = pd.DataFrame(columns=['CAPTURA', 'FUENTE', 'ORIGEN', 'DESTINO', 'SALIDA', 'LLEGADA', 'PRECIO', "URL", "AEROLINEA"])
        df_vuelta = pd.DataFrame(columns=['CAPTURA', 'FUENTE', 'ORIGEN', 'DESTINO', 'SALIDA', 'LLEGADA', 'PRECIO', "URL", "AEROLINEA"])
        try:
            url = 'https://www.ryanair.com/es/es/trip/flights/select?adults=1&teens=0&children=0&infants=0&dateOut='+texto_fecha_ida+'&dateIn='+texto_fecha_vuelta+'&originIata='+j+'&destinationIata='+i+'&isConnectedFlight=true&isReturn=true&discount=0&promoCode=&tpAdults=1&tpTeens=0&tpChildren=0&tpInfants=0&tpStartDate='+texto_fecha_ida+'&tpEndDate='+texto_fecha_vuelta+'&tpOriginIata='+j+'&tpDestinationIata='+i+'&tpIsConnectedFlight=true&tpIsReturn=true&tpDiscount=0&tpPromoCode='
            navegador.implicitly_wait(20)
            navegador.get(url) 
            sleep(randint(3,6))
            source = navegador.page_source
            soup = BeautifulSoup(source)
            soup2 = soup.find("button", {'data-ref':texto_fecha_ida})
            soup3 = soup.find("button", {'data-ref':texto_fecha_vuelta})
            try:
                precio1 = soup2.find("span", class_="price__integers carousel-date-price--selected").text.strip()
                precio2 = soup3.find("span", class_="price__integers carousel-date-price--selected").text.strip()
                if int(precio1) >0 and int(precio2) >0:
                    soup_ida = soup.find("div", class_="ng-tns-c31-8 ng-star-inserted")
                    soup_vuelta = soup.find("div", class_="ng-tns-c31-11 ng-star-inserted")
                    datos = GetVuelos(soup_ida,url,j,i,texto_fecha_ida)
                    df_ida = df_ida.append(datos)
                    datos = GetVuelos(soup_vuelta,url,j,i,texto_fecha_vuelta)
                    df_vuelta = df_vuelta.append(datos)
            except:
                continue
        except:
            print('No existe vuelo')
            continue
        finally:
            for fila_ida in range(len(df_ida)):
                for fila_vuelta in range(len(df_vuelta)):
                    df_vuelos = df_vuelos.append({'CAPTURA': df_ida['CAPTURA'].iat[fila_ida], 
                                                  "FUENTE": "Ryanair", 
                                                  "ORIGEN":j, 
                                                  "DESTINO":i, 
                                                  "SALIDA_IDA":df_ida["SALIDA"].iat[fila_ida], 
                                                  "LLEGADA_IDA":df_ida["LLEGADA"].iat[fila_ida], 
                                                  "SALIDA_VUELTA":df_vuelta["SALIDA"].iat[fila_vuelta], 
                                                  "LLEGADA_VUELTA":df_vuelta["LLEGADA"].iat[fila_vuelta],
                                                  "PRECIO":df_ida["PRECIO"].iat[fila_ida]+df_vuelta["PRECIO"].iat[fila_vuelta],
                                                  "URL": df_ida["URL"].iat[fila_ida],
                                                  "AEROLINEA":'Ryanair'}, 
                                                  ignore_index = True) 
            sleep(randint(3,6))
            
    fecha_ida += incrementoDia
    fecha_vuelta += incrementoDia
    texto_fecha_ida = fecha_ida.strftime('%Y-%m-%d')
    texto_fecha_vuelta = fecha_vuelta.strftime('%Y-%m-%d')
navegador.quit()

Wall time: 16h 42min 32s


**GUARDADO DE LOS DATOS**

Para finalizar, se guarda el archivo como data_ryanair.csv o se sobreescribe si ya existe. Ruta relativa "../../data/data_ryanair.csv".

In [59]:
from os import path
if path.exists('../../data/data_ryanair.csv'):
    df_vuelos.to_csv('../../data/data_ryanair.csv', mode='a', header=False)
else:
    df_vuelos.to_csv("../../data/data_ryanair.csv")